DataLoader

In [ ]:
# * build DATALODER * #
train_loader = build_train_loader(cfg)
val_loader = build_val_loader(cfg)

buildDataloader.py

In [ ]:
import importlib

import torch
import torch.distributed as dist

def build_train_loader(cfg):

    dataModule = importlib.import_module('data.'+cfg.DATA.SET)
    Dataset = getattr(dataModule, 'Dataset')
    train_data = Dataset(cfg, mode='train')

    if dist.is_initialized():
        from torch.utils.data.distributed import DistributedSampler
        sampler = DistributedSampler(train_data)
    else:
        sampler =  None

    data_loader = torch.utils.data.DataLoader(train_data,
                            num_workers= cfg.SYS.WORKERS,
                            batch_size=cfg.TRAIN.BATCH_SIZE//len(cfg.SYS.GPUS),
                            shuffle=cfg.DATA.SHUFFLE,
                            pin_memory=cfg.SYS.PIN_MEMORY,
                            drop_last=cfg.DATA.DROP_LAST,
                            sampler=sampler)

    return data_loader

def build_val_loader(cfg):

    dataModule = importlib.import_module('data.'+cfg.DATA.SET)
    Dataset = getattr(dataModule, 'Dataset')
    val_data = Dataset(cfg, mode='val')

    if dist.is_initialized():
        from torch.utils.data.distributed import DistributedSampler
        sampler = DistributedSampler(val_data)
    else:
        sampler =  None

    data_loader = torch.utils.data.DataLoader(val_data,
                                            num_workers=cfg.SYS.WORKERS,
                                            batch_size= cfg.TRAIN.BATCH_SIZE_VAL//len(cfg.SYS.GPUS),
                                            shuffle=cfg.DATA.SHUFFLE,
                                            pin_memory=cfg.SYS.PIN_MEMORY,
                                            drop_last=cfg.DATA.DROP_LAST,
                                            sampler=sampler)

    return data_loader
    
#! TODO
def build_test_loader(cfg):

    dataModule = importlib.import_module('data.'+cfg.DATA.SET)
    Dataset = getattr(dataModule, 'Dataset')
    test = Dataset(cfg, mode='test')

    if dist.is_initialized():
        from torch.utils.data.distributed import DistributedSampler
        sampler = DistributedSampler(val_data)
    else:
        sampler =  None

    data_loader = torch.utils.data.DataLoader(test_data,
                                            num_workers=4,
                                            batch_size=1,
                                            shuffle=False,
                                            pin_memory=True,
                                            sampler=None)


if __name__=='__main__':
    import torchvision
    import matplotlib.pyplot as plt
    import matplotlib.colors as clr
    from data import transform

    root='data/cityscapes/'
    # print(root)
    value_scale = 255
    mean = [0.485, 0.456, 0.406]
    mean = [item * value_scale for item in mean]
    std = [0.229, 0.224, 0.225]
    std = [item * value_scale for item in std]

    dataset = Cityscapes(root,transforms=train_transform)

    img, label =dataset.__getitem__(0)
    print(img.dtype)
    print(label.shape)
    fig_in = plt.figure()
    ax = fig_in.add_subplot(1,2,1)
    ax.imshow(img)
    ax = fig_in.add_subplot(1,2,2)
    ax.imshow(label)
    plt.show()


DataSet Class

Build Dataloader

Data Transform